In [1]:
# pre-install required libraries
import warnings
#%pip install -upgrade pip
%pip install spacy
%pip install ipywidgets
#%pip install -U jupyter

# suppress user warnings during execution
warnings.filterwarnings(action='ignore', category=UserWarning)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Using the AAT Vocabulary Annotator
## Introduction
The AatVocabularyAnnotator class includes a series of custom spaCy pipeline components to perform *vocabulary-based* lookup matching on terms originating from the Getty Art &amp; Architecture Thesaurus (AAT). These components are based on existing monolingual (English) terms so they can currently be used to identify and tag English language terms. The following entity types can be tagged by the AatVocabularyAnnotator. Entity types can be selectively included/excluded:

| Entity Type   | Description   | Examples  |
|---------------|---------------| ----------|
| ACTIVITY      | Terms from the [AAT 'Activities' facet](http://vocab.getty.edu/aat/300264090)   | *examples..* |
| AGENT         | Terms from the [AAT 'Agents' facet](http://vocab.getty.edu/aat/300264089)  | *examples..* |
| MATERIAL      | Terms from the [AAT 'Materials' facet](http://vocab.getty.edu/aat/300264091) | *examples..* |
| OBJECT        | Terms from the [AAT 'Objects' facet](http://vocab.getty.edu/aat/300264092) | *examples..* |
| STYLEPERIOD   | Terms from the [AAT 'Styles &amp; Periods' facet](http://vocab.getty.edu/aat/300264088) | *examples..* |

In [3]:
# simple example using AatVocabularyAnnotator on a passage of text
from rematch2.AatVocabularyAnnotator import AatVocabularyAnnotator
from IPython.display import display, HTML

# example test text from https://doi.org/10.5284/1100093
test_text = """This collection comprises site data (images, a report, a project database and GIS data) from an archaeological excavation undertaken by Cotswold Archaeology between January and February 2020 at Lydney B Phase III, Archers Walk, Lydney, Gloucestershire. An area of 0.6ha was excavated within this phase (Phase III) of a wider development area.
Aside from three residual flints, none closely datable, the earliest remains comprised a small assemblage of Roman pottery and ceramic building material, also residual and most likely derived from a Roman farmstead found immediately to the north within the Phase II excavation area. A single sherd of Anglo-Saxon grass-tempered pottery was also residual.
The earliest features, which accounted for the majority of the remains on site, relate to medieval agricultural activity focused within a large enclosure. There was little to suggest domestic occupation within the site: the pottery assemblage was modest and well abraded, whilst charred plant remains were sparse, and, as with some metallurgical residues, point to waste disposal rather than the locations of processing or consumption. A focus of occupation within the Rodley Manor site, on higher ground 160m to the north-west, seems likely, with the currently site having lain beyond this and providing agricultural facilities, most likely corrals and pens for livestock. Animal bone was absent, but the damp, low-lying ground would have been best suited to cattle. An assemblage of medieval coins recovered from the subsoil during a metal detector survey may represent a dispersed hoard.
"""

# required output format options: html|csv|json|dataframe|doc
# 'html' returns inline markup for visualising annotations in context
# 'dataframe' useful for visualising tabular data in python notebook
# 'csv' and 'json' are useful textual interchange formats
# 'doc' returns the spaCy document object for further processing
output_format = "html"   

# entity types may be selectively included/excluded and reordered
# e.g. ["OBJECT", "MONUMENT", "MATERIAL"]
annotator = AatVocabularyAnnotator(entity_types=[
    "ACTIVITY", 
    "AGENT", 
    "MATERIAL",
    "OBJECT", 
    "STYLEPERIOD"
])

# process example text and display the results in required output format
results = annotator.annotateText(input_text=test_text, format=output_format)
display(results)



None

In [4]:
# UI to test AatVocabularyAnnotator on a range of example texts
import ipywidgets as widgets
from IPython.display import display, HTML
from rematch2.AatVocabularyAnnotator import AatVocabularyAnnotator
from test_examples_english import test_examples_english

# TODO - choose these as checkboxes in UI
annotator = AatVocabularyAnnotator(entity_types=[
    "ACTIVITY", "AGENT", "MATERIAL",  
    "OBJECT", "STYLEPERIOD" 
])

def run(btn):
    # clear any previous output
    output.clear_output(wait=True)
   
    # get the test text for the for the chosen id    
    selected_test = next(
        filter(lambda test: test.get("id", "") == dropdown_tests.value, test_examples_english), None)
    if(selected_test):
        # get annotation results
        output_format = dropdown_format.value
        results = annotator.annotateText(
            input_text=selected_test.get("text", ""), format=output_format)
        # display annotation results
        with output:
            if(output_format == "html"):
                display(HTML(results))
            else:
                display(results)
        
# define test selector dropdown UI component
dropdown_tests = widgets.Dropdown(
    options=[[test.get("source",""), test.get("id","")]
             for test in test_examples_english],
    #value="en",
    description='Test:',
    disabled=False
)

# define output format dropdown UI component
dropdown_format = widgets.Dropdown(
    options=[
        ["HTML", "html"],
        ["Tabular", "dataframe"]
    ],
    value="html",
    description="Format:",
    disabled=False
)

# define and display other UI components
button_go = widgets.Button(description="Go")
input = widgets.HBox([dropdown_tests, dropdown_format, button_go])
output = widgets.Output(layout=widgets.Layout(
    overflow='scroll', border='1px solid black', height='500px'))
display(input, output)

# what to do when the button is clicked
button_go.on_click(run)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…